In [1]:
import pandas as pd

def get_lower_approximation(df, decision_col, condition_cols, decision_val):
    """
    计算决策属性值为decision_val时，条件属性的下近似集。
    df: 数据框，包含条件属性和决策属性。
    decision_col: 决策属性列名。
    condition_cols: 条件属性列名列表。
    decision_val: 决策属性值。
    返回一个数据框，包含满足条件属性下近似集的记录。
    """
    # 筛选决策属性为decision_val的记录
    subset = df[df[decision_col] == decision_val][condition_cols + [decision_col]]

    # 对条件属性进行去重
    subset = subset.drop_duplicates(subset=condition_cols)

    # 将满足条件属性的记录合并为下近似集
    lower_approximation = subset[condition_cols]

    return lower_approximation

def get_key_attribute(df, decision_col, condition_cols):
    """
    选择关键属性，即对于每个决策类别，选择一个最能区分不同条件属性的属性。
    df: 数据框，包含条件属性和决策属性。
    decision_col: 决策属性列名。
    condition_cols: 条件属性列名列表。
    返回一个字典，包含每个决策类别的关键属性。
    """
    # 根据决策属性将记录分组
    groupby_decision = df.groupby(decision_col)

    key_attribute = {}

    # 对于每个决策类别，选择一个最能区分不同条件属性的属性
    for decision_val, group in groupby_decision:
        max_score = -1
        best_attribute = None

        # 对于每个属性，计算其得分，并选择得分最高的属性作为关键属性
        for attribute in condition_cols:
            # 计算当前属性的得分
            score = 0
            for value in set(group[attribute]):
                subset = get_lower_approximation(group, decision_col, [attribute], decision_val)
                score += len(subset)

            # 更新得分最高的属性
            if score > max_score:
                max_score = score
                best_attribute = attribute

        key_attribute[decision_val] = best_attribute

    return key_attribute

def generate_rules(df, decision_col, condition_cols):
    """
    生成LEM2算法的决策规则。
    df: 数据框，包含条件属性和决策属性。
    decision_col: 决策属性列名。
    condition_cols: 条件属性列名列表。
    返回一个列表，包含所有生成的决策规则。
    """
    # 根据决策属性将记录分组
    groupby_decision = df.groupby(decision_col)

    rules = []

    # 对于每个决策类别，生成一个简单的决策规则
    for decision_val, group in groupby_decision:
        key_attribute = get_key_attribute(group, decision_col, condition_cols)
        rule = {decision_col: decision_val}

        for attribute, value in key_attribute.items():
            subset = get_lower_approximation(group, decision_col, [attribute], decision_val)
            rule[attribute] = set(subset[value]).pop()

        rules.append(rule)

    return rules


In [3]:
import pandas as pd

# 加载示例数据集
df = pd.read_csv('data/Soybean/soybean.csv')

# 选择条件属性和决策属性
condition_cols = ['f4', 'f5', 'f6']
decision_col = 'target'

# 生成决策规则
rules = generate_rules(df, decision_col, condition_cols)

# 输出所有决策规则
for rule in rules:
    print(rule)


KeyError: "['2-4-d-injury'] not in index"